In [1]:
import os
import numpy as np
import pandas as pd
from nilearn import plotting
from nilearn import image
from nilearn import datasets
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

In [2]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm') ## bringing in atlas
atlas_filename = dataset.maps
labels= dataset.labels ## contains ROIs
masker = NiftiLabelsMasker(labels_img = atlas_filename, standardize=True) ## extract signals

In [3]:
# Set the folder with documents for text extraction as the working directory
working_dir = r"/Users/rebeccaehrenkranz/Desktop/HCP_HP2000_CLEAN/"

# Validate directory
if not os.path.exists(working_dir):
    raise Exception ("Working directory '%s' not found!" % (working_dir))
    
# Iterate through all files in folder and subfolders
for root, dirs, files in os.walk(working_dir):
    for filename in files:
        
        # Check if file type is NIFTI 
        if os.path.splitext(filename)[1] == '.nii':
            participant_id = filename.split("_")[0]
            
            # go from nifti to signal time series in numpy array
            time_series = masker.fit_transform(os.path.join(working_dir, filename))
            
            # Creating the correlation matrix for each participant transforming time series data
            correlation_measure = ConnectivityMeasure(kind='correlation')
            correlation_matrix = correlation_measure.fit_transform([time_series])[0]
            
            # Create a subdirectory called 'txt' for plain text files
            output_dir = "/Users/rebeccaehrenkranz/Documents/hcp_correlation_matrices/"

            try:
                os.stat(output_dir)
            except:
                os.mkdir(output_dir)
            
            # Creating csv exports of correlation matrices for each participant
            df = pd.DataFrame(correlation_matrix)
            df.index = labels[1:  ]
            output_filename = '%s_correlation_matrix.csv' % participant_id
            df.to_csv((output_dir + output_filename), header=labels[1: ])
            